In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
data = pd.read_csv('projet 2.csv')

C:\Users\chabc\AppData\Local\Temp\ipykernel_14736\1692616215.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('projet 2.csv')


In [3]:
nouvelle_data = data.loc[:, ['Produit', 'Desc-plainte-C']]

In [4]:
nouvelle_data_non_null = nouvelle_data.dropna(subset=['Desc-plainte-C'])

In [5]:
# Prétraitement des données
max_words = 10000  # Nombre maximal de mots à prendre en compte
max_len = 50  # Longueur maximale des séquences
#utilisé pour convertir le texte en séquences d'entiers. 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(nouvelle_data_non_null['Desc-plainte-C'])
# ikharej kol aprtie wahdha 
sequences = tokenizer.texts_to_sequences(nouvelle_data_non_null['Desc-plainte-C'])
# Les séquences plus courtes sont remplies de zéros à gauche, et les séquences plus longues sont tronquées.
X = pad_sequences(sequences, maxlen=max_len)
y = pd.get_dummies(nouvelle_data_non_null['Produit'])

In [6]:
# Fractionner les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Créer le modèle RNN Réseau de Neurones Récurrents
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=64))  # Supprimez input_length
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(y.columns), activation='softmax'))


In [8]:
# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 73s 25ms/step - accuracy: 0.4787 - loss: 1.5113 - val_accuracy: 0.6970 - val_loss: 0.9505
Epoch 2/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 87s 30ms/step - accuracy: 0.7156 - loss: 0.8970 - val_accuracy: 0.7377 - val_loss: 0.8294
Epoch 3/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.7621 - loss: 0.7494 - val_accuracy: 0.7443 - val_loss: 0.8116
Epoch 4/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 84s 29ms/step - accuracy: 0.7887 - loss: 0.6641 - val_accuracy: 0.7432 - val_loss: 0.8119
Epoch 5/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 86s 30ms/step - accuracy: 0.8100 - loss: 0.5976 - val_accuracy: 0.7417 - val_loss: 0.8289
Epoch 6/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 83s 29ms/step - accuracy: 0.8229 - loss: 0.5527 - val_accuracy: 0.7380 - val_loss: 0.8519
Epoch 7/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 83s 29ms/step - accuracy: 0.8360 - loss: 0.5221 - val_accuracy: 0.7338 - val_loss: 0.8868
Epoch 8/10
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 93s 32ms/step - accuracy: 0.8440 -

In [10]:
# Évaluer la performance du modèle
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy du modèle:', accuracy)

717/717 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7236 - loss: 0.9873
Accuracy du modèle: 0.7269081473350525


In [11]:
# Identifier les lignes avec des valeurs manquantes dans 'Desc-plainte-C'
missing_data = nouvelle_data[nouvelle_data['Desc-plainte-C'].isna()]

In [12]:
# Utiliser la colonne 'Produit' pour prédire les valeurs manquantes
predicted_missing = missing_data['Produit']

In [13]:
# Remplacer les valeurs manquantes dans 'Desc-plainte-C' avec les valeurs prédites
nouvelle_data.loc[missing_data.index, 'Desc-plainte-C'] = predicted_missing

In [14]:
nouvelle_data.head(8963)

,Produit,Desc-plainte-C
0,Consumer Loan,Consumer Loan
1,Bank account or service,Bank account or service
2,Bank account or service,Bank account or service
3,Bank account or service,Bank account or service
4,Mortgage,Mortgage
...,...,...
8958,Credit card,Credit card
8959,Bank account or service,Bank account or service
8960,Mortgage,Mortgage
8961,Mortgage,Mortgage


In [15]:
nouvelle_data = nouvelle_data.loc[: , ["Desc-plainte-C"]]

In [16]:
nouvelle_data.to_csv("dataset_rempli_specific_Desc-plainte-C.csv", index=False)

In [17]:
nouvelle_data.to_excel("dataset_rempli_specific_Desc-plainte-C.xlsx", index=False)